## Графовые нейронные сети (GNNs)

Графовые нейронные сети представляют собой класс нейронных сетей, специально разработанный для работы с графовыми структурами данных. Они могут моделировать зависимости и взаимодействия между сущностями в графе, позволяя учитывать контекст и структуру данных при выполнении задач.

В GNN каждая вершина графа обновляется на основе её соседей, что позволяет модели учить представления вершин, учитывая их окружение. Такие модели широко применяются в задачах, связанных с анализом социальных сетей, биологических сетей, рекомендательных систем и графовых знаний.

## Graph embedding
Эмбеддинги в контексте графовых нейронных сетей представляют собой векторные представления вершин графа. Они извлекаются из GNN и представляют сущности в пространстве низкой размерности. Эти векторы обладают свойством сохранять структурную информацию о графе и могут использоваться для различных задач, таких как предсказание отношений или классификация вершин.

## Тестирование и Валидация

Тестирование и валидация важны для оценки работы графовых нейронных сетей.

Тестирование позволяет оценить обобщающую способность модели на новых данных, которые не участвовали в обучении. Это важно для проверки, насколько хорошо модель обобщает свои знания на новые сценарии.

Валидация используется для подбора оптимальных гиперпараметров модели. Настройка параметров происходит на основе производительности модели на отложенной валидационной выборке.

### Установка зависимостей

Для работы с графовыми нейронными сетями будем использовать [Pykeen](https://pykeen.readthedocs.io/en/stable/). Необходимо импортировать соответствующую библиотеку. В данном шаге мы устанавливаем Pykeen. Этот шаг включает установку библиотеки Pykeen с использованием команды !pip install pykeen. Pykeen предоставляет реализацию различных моделей графовых знаний и инструменты для работы с ними.

In [1]:
# Установка Pykeen
!pip install pykeen

Далее, мы импортируем библиотеку Pykeen для использования её функциональности в нашем коде.

In [2]:
import pykeen

После выполнения этого шага, мы готовы к использованию Pykeen для создания, обучения и оценки моделей графовых знаний.

## Взаимодействие с графовым датасетом и описание данных

В данной работе мы будем использовать графовый датасет [Nations](https://paperswithcode.com/dataset/nations), предоставляющий информацию о связях между стран и их политических отношений. Этот небольшой датасет хорошо подходит для погружения в задачу предсказания отношений между сущностями.


Импорт необходимого модуля Pykeen. Мы импортируем модуль datasets из Pykeen для работы с графовыми датасетами.

In [3]:
from pykeen.datasets import CoDExSmall

INFO:pykeen.utils:Using opt_einsum


Загружаем графовый датасет Nations из Pykeen. В результате выполнения этой команды будет создан объект, представляющий графовый датасет.

In [4]:
# Загрузка графового датасета CoDExSmall
CoDExSmall_dataset = CoDExSmall()

Мы можем вывести описание датасета, чтобы получить информацию о количестве сущностей, отношений и других характеристиках.

In [5]:
# Просмотр описания датасета
print(CoDExSmall_dataset)

CoDExSmall(training_path="/root/.data/pykeen/datasets/codexsmall/train.txt", testing_path="/root/.data/pykeen/datasets/codexsmall/test.txt", validation_path="/root/.data/pykeen/datasets/codexsmall/valid.txt")


#### Получение тренировочных данных
Мы можем получить тренировочные факты, представленные в виде троек сущность-отношение-сущность.
Таким образом, после выполнения этих шагов, у нас есть загруженный графовый датасет, и мы можем использовать его для обучения и тестирования моделей графовых знаний в Pykeen.

In [6]:
# Получение тренировочных фактов (троек сущность-отношение-сущность)
training_data = CoDExSmall_dataset.training.mapped_triples
print(training_data[:5])

tensor([[  1,  23, 263],
        [  1,  28, 264],
        [  1,  31, 222],
        [  1,  31, 452],
        [  1,  31, 576]])


## Описание базовой задачи

В данном контексте базовой задачей является предсказание отношений между сущностями в графовом датасете. Каждая тройка в графе представляет отношение между двумя сущностями. Например, в социальной сети, где сущности - пользователи и страницы, отношения могут быть "пользователь подписан на страницу".

In [7]:
# Импорт необходимых модулей Pykeen
from pykeen.pipeline import pipeline


#### Выбор модели
Мы выбираем модель для решения задачи предсказания отношений. В данном случае, мы используем модель [TransE](https://paperswithcode.com/method/transe), одну из популярных моделей для этой задачи.

#### Выбор датасета
Мы указываем графовый датасет, с которым будем работать. В данном примере - CoDExSmall_dataset, который содержит данные о связях в между странами.

#### Количество эпох обучения
Задаем количество эпох обучения (итераций по всему датасету).

Проверяем наличие GPU устройства и вызываем функцию pipeline(), которая запускает процесс обучения модели, тестирования и валидации.

In [8]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [9]:
# Задаем параметры для pipeline
pipeline_results = pipeline(
    model="TransE",      # Выбираем модель (TransE - одна из моделей для предсказания отношений)
    dataset=CoDExSmall_dataset,     # Выбираем графовый датасет (Nations)
    training_kwargs=dict(num_epochs=20),   # Количество эпох обучения
    device=device
)

INFO:pykeen.pipeline.api:Using device: cuda


Training epochs on cuda:0:   0%|          | 0/20 [00:00<?, ?epoch/s]

Training batches on cuda:0:   0%|          | 0/129 [00:00<?, ?batch/s]

Training batches on cuda:0:   0%|          | 0/129 [00:00<?, ?batch/s]

Training batches on cuda:0:   0%|          | 0/129 [00:00<?, ?batch/s]

...

Training batches on cuda:0:   0%|          | 0/129 [00:00<?, ?batch/s]

Training batches on cuda:0:   0%|          | 0/129 [00:00<?, ?batch/s]

INFO:pykeen.evaluation.evaluator:Starting batch_size search for evaluation now...
INFO:pykeen.evaluation.evaluator:Concluded batch_size search with batch_size=2048.


Evaluating on cuda:0:   0%|          | 0.00/1.83k [00:00<?, ?triple/s]

INFO:pykeen.evaluation.evaluator:Evaluation took 0.36s seconds



Этот код инициирует обучение модели на выбранном датасете, используя выбранную архитектуру (TransE) и указанные параметры. После завершения обучения, результаты выводятся, и можно провести анализ производительности модели.


Печатаем результаты, которые могут включать в себя метрики производительности, потери на тренировочных данных, а также результаты на тестовом и валидационном наборах данных.

In [10]:
# Выводим результаты
print(pipeline_results)

PipelineResult(random_seed=3632653504, model=TransE(
  (loss): MarginRankingLoss(
    (margin_activation): ReLU()
  )
  (interaction): TransEInteraction()
  (entity_representations): ModuleList(
    (0): Embedding(
      (_embeddings): Embedding(2034, 50)
    )
  )
  (relation_representations): ModuleList(
    (0): Embedding(
      (_embeddings): Embedding(42, 50)
    )
  )
  (weight_regularizers): ModuleList()
), training=TriplesFactory(num_entities=2034, num_relations=42, create_inverse_triples=False, num_triples=32888, path="/root/.data/pykeen/datasets/codexsmall/train.txt"), training_loop=<pykeen.training.slcwa.SLCWATrainingLoop object at 0x7e4e1c003b20>, losses=[0.9790182760519575, 0.7734395057656044, 0.6474137680475102, 0.5447765533314195, 0.46821345839389533, 0.39826628542685694, 0.3500460069770961, 0.3127305399077807, 0.2842099834089131, 0.26326088168362316, 0.24514774989712146, 0.22924301501854447, 0.21639203771140225, 0.20459344648113548, 0.18946241760669752, 0.18495988071888

### Triple Scoring

Импортируем [predict_triples](https://pykeen.readthedocs.io/en/stable/api/pykeen.predict.predict_triples.html)

In [11]:
from pykeen.predict import predict_triples

Рассчитаем оценки для всех validation троек из набора данных, на котором мы обучали модель.

In [12]:
pack = predict_triples(model=pipeline_results.model, triples=CoDExSmall_dataset.validation)
df = pack.process(factory=pipeline_results.training).df

Показаны тройки, получившие наивысший score

In [13]:
df.nlargest(n=5, columns="score")

,head_id,head_label,relation_id,relation_label,tail_id,tail_label,score
1448,1673,Q668,34,P530,2008,Q96,-5.760135
1167,1299,Q408,34,P530,1673,Q668,-5.770810
231,260,Q148,34,P530,917,Q258,-5.984328
1202,1329,Q42229,22,P27,1030,Q30,-5.999054
817,921,Q2599,9,P136,1473,Q484641,-6.033972


В рамках самостоятельной работы попробуйте обучить и протестировать модель на любом другом датасете доступном на https://pykeen.readthedocs.io/en/stable/reference/datasets.html.